In [1]:
!pip install -qU xpander-sdk llama-index llama-index-llms-openai python-dotenv openai


In [1]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from os import environ
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = environ["OPENAI_API_KEY"]
XPANDER_API_KEY = environ["XPANDER_API_KEY"]
XPANDER_AGENT_ID = environ["XPANDER_AGENT_ID"]

In [3]:
llm = OpenAI(model="gpt-4o")
llama_agent = ReActAgent.from_llm(llm=llm, verbose=True)
llama_agent.chat("What is the weather in Tokyo?")

Added user message to memory: What is the weather in Tokyo?


AgentChatResponse(response="I'm unable to provide real-time weather updates. For the current weather in Tokyo, I recommend checking a reliable weather website or app.", sources=[], source_nodes=[], is_dummy_stream=False, metadata=None)

In [3]:
from openai import OpenAI
from xpander_sdk import XpanderClient
xpander_client = XpanderClient(api_key=XPANDER_API_KEY, organization_id="")
xpander_agent = xpander_client.agents.get(agent_id=XPANDER_AGENT_ID)

xpander_agent.invoke_agent("Summarize the profile of david twizer from xpander.ai")
openai_client = OpenAI(api_key=OPENAI_API_KEY)
xpander_agent.memory.initialize(input=xpander_agent.execution.input_message,instructions=xpander_agent.instructions)
while not xpander_agent.is_finished():
    print("-"*100)
    response = openai_client.chat.completions.create(
                model= 'gpt-4o',
                messages=xpander_agent.memory.retrieve_messages(),
                tools=xpander_agent.get_tools(),
                tool_choice="auto",
                temperature=0.0
        )
            
    xpander_agent.memory.add_messages(response.model_dump())
    tool_calls = XpanderClient.extract_tool_calls(llm_response=response.model_dump())
    tool_responses = xpander_agent.run_tools(tool_calls=tool_calls)
    xpander_agent.memory.add_tool_call_results(tool_responses)
# result (re fetch execution result)
# IMPORTANT: LAST TOOL IS xpfinish-agent-execution-finished WHICH IS AGENT-END with PARSING and may be slower due to inference times
execution_result = xpander_agent.execution.fetch(agent=xpander_agent,execution_id=xpander_agent.execution.id)
print("status", execution_result.status)
print("result", execution_result.result)

b'\x1b\x1b!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!\x1b\x1b\n'
b'\x1b\x1b!!                                                                                                                      !!\x1b\x1b\n'
b'\x1b\x1b!!  This software has not been tested with node v23.4.0.                                                                !!\x1b\x1b\n'
b'\x1b\x1b!!  Should you encounter odd runtime issues, please try using one of the supported release before filing a bug report.  !!\x1b\x1b\n'
b'\x1b\x1b!!                                                                                                                      !!\x1b\x1b\n'
b'\x1b\x1b!!  This software is currently running on node v23.4.0.                                                                 !!\x1b\x1b\n'
b'\x1b\x1b!!  As of the current release of this software, supported node releases are:                                            !!\x1b

----------------------------------------------------------------------------------------------------


b'{"stdout":"cnVubmluZyB0b29sIExpbmtlZEluUHJvZmlsZVNlcnZpY2VTZWFyY2hQcm9maWxlc0J5Q3JpdGVyaWEgb24gYWdlbnQgODYwZmM3MjUtOTcyOC00NmY3LThiMGItYzQ1Nzc0ZjQ0ZjE5IHdpdGggZXhlY3V0aW9uIDU1ZjdlMzJhLTI3NzAtNGJiNy1iNmJjLTg4OTA2MGEzZmZhNAo="}\n'


----------------------------------------------------------------------------------------------------


BadRequestError: Error code: 400 - {'error': {'message': "Invalid parameter: messages with role 'tool' must be a response to a preceeding message with 'tool_calls'.", 'type': 'invalid_request_error', 'param': 'messages.[4].role', 'code': None}}